# Air quality impact on hives survival 

The dataset is from Nectar 1.19 and cover hives from 2020 to 2023. The 2023 season is currently censored in terms of winter mortality but you can analyse the acute effect (which I didn't include in the blog post but I did have a look at the effect on the summer mortality and the effect are even stronger). I also shared the complete daily history for all the yards in the dataset for air quality , ndvi and weather that was scrapped from Airpyllution package, meteostats package and earth engine. Finally, you will find a file called 'ops_meta.csv' which contains the region of the given operations. 

The file are found on this google drive: [here](https://drive.google.com/drive/u/0/folders/1UZ3_jzMRdaQEaIR1TzPqEWTb0azECStB)


***
### SETUP

In [1]:
import os 
import ast
from datetime import datetime, date,timezone, timedelta

import pandas as pd 
import numpy as np
from collections import Counter

import seaborn as sns 
import matplotlib.pyplot as plt 
import matplotlib.ticker as mtick
%matplotlib inline 

from sklearn.preprocessing import MinMaxScaler
from scipy.stats import skew, kurtosis
from pymer4.models import Lmer
from lifelines import KaplanMeierFitter, CoxPHFitter
from sklearn.model_selection import train_test_split
from sksurv.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sksurv.ensemble import RandomSurvivalForest
from sklearn.model_selection import RandomizedSearchCV

from sksurv.metrics import (
    concordance_index_censored,
    concordance_index_ipcw,
    cumulative_dynamic_auc,
    integrated_brier_score,
)

from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings("ignore")

NECTAR_PALETTE = "blend:#D8A348,#1D1D1D"
sns.set_palette(NECTAR_PALETTE)

ROOT_PATH = "../data"
SEASONS_INCLUDED = [2021, 2022, 2023]
FEATURES_MONTH = [5,6,7,8]
RUN_PREPROCESS = True

# Don't change these values 
START_SEASON_MONTH = 6
START_SEASON_DAY = 15
END_SEASON_MONTH = 11
END_SEASON_DAY = 15

# OPS USED 
OPS = (36, 51, 55, 69, 83, 87, 89, 153, 159, 160, 161, 167, 192, 193, 194, 195, 199, 205, 207, 208, 210, 212, 218, 219, 220, 221)

In [2]:
from aqi.data.utils import timestamp_to_date, try_check_month
from aqi.vizualisation.utils import plot_survival_bar_per_state

In [3]:

if RUN_PREPROCESS:
    d = pd.read_csv(os.path.join(ROOT_PATH,"raw_dev.csv"))
    d['death_date'] = pd.to_datetime(d['death_date'],format="mixed")
    d['creation_date'] = pd.to_datetime(d['creation_date'],format="mixed")

    for season in SEASONS_INCLUDED:
    
        season_end = date(season,END_SEASON_MONTH,END_SEASON_DAY)
        last_season_end = date(season-1,END_SEASON_MONTH,END_SEASON_DAY)
        season_start = date(season,START_SEASON_MONTH,START_SEASON_DAY)
    
        next_season_start = date(season+1,START_SEASON_MONTH,START_SEASON_DAY)
    
        state_logs = d.copy()
        state_logs["death_date"]= state_logs["death_date"].fillna(next_season_start)
        state_logs["death_date"] = pd.to_datetime(state_logs["death_date"]).dt.tz_localize(None)
        state_logs["creation_date"] = pd.to_datetime(state_logs["creation_date"]).dt.tz_localize(
            None
        )
       
        created_during_season = (state_logs["creation_date"].dt.date < season_end) 
        # Died after the season start but was alive before
        died_after_season_start = state_logs["death_date"].dt.date > season_start
        # Died before the next season 
        died_before_next_season_start = state_logs["death_date"].dt.date <= next_season_start #+ timedelta(days=BUFFER)
        # Exclude hive created betwween season start and that die before also 
        didnt_see_summer = ~((state_logs.apply(lambda x: x["death_date"].month in [1,2,3,4,5] ,axis=1)) & (state_logs["creation_date"].dt.date > season_end))
    
        
        subset_state = state_logs.loc[created_during_season & died_after_season_start & died_before_next_season_start & didnt_see_summer]
        subset_state['season'] = season
        # Also replace death date > then end_of_exp because otherwise bias model (age)
        subset_state.loc[pd.to_datetime(subset_state['death_date']).dt.date > next_season_start, 'death_date'] = next_season_start
        
        if season == SEASONS_INCLUDED[0]:
            data = subset_state 
        else:
            data = pd.concat([data,subset_state],axis=0)
    data.reset_index(drop=True,inplace=True)
    data.to_csv(os.path.join(ROOT_PATH,"merged_dev.csv"),index=False)

***
### LOADING DATA

In [4]:
if RUN_PREPROCESS:
    operation_meta = pd.read_csv(os.path.join(ROOT_PATH,"ops_meta.csv"))
    aqi = pd.read_csv(os.path.join(ROOT_PATH,"aqi_dev.csv"))
    weather = pd.read_csv(os.path.join(ROOT_PATH,"weather_dev.csv"))
    ee = pd.read_csv(os.path.join(ROOT_PATH,"ee_dev.csv"))
    ee.drop("ee-image",axis=1,inplace=True)
    ee.dropna(subset=["ndvi"],inplace=True)
    
    mov = pd.read_csv(os.path.join(ROOT_PATH,"mouvements.csv"))

    data = pd.read_csv(os.path.join(ROOT_PATH,"merged_dev.csv"))
else:
    data = pd.read_csv(os.path.join(ROOT_PATH,"preprocess.csv"))
data['death_date'] = pd.to_datetime(data['death_date'], format='mixed')
data = data.loc[data['season'].isin(SEASONS_INCLUDED)]

In [5]:
# sns.set_palette("tab10")
# for ops in d.operation_id.unique():
#     if ops in [55,161,69]:
#         sub = d.loc[d['operation_id']==ops]
#         sns.histplot(sub.death_date.dt.month, label=str(ops))
# plt.legend()

In [6]:
sns.set_palette(NECTAR_PALETTE)

In [7]:
if RUN_PREPROCESS:
    print(weather.isnull().mean()) # For some reason atmost pressure is not getting scrapped 

time       0.000000
yard_id    0.000000
tavg       0.001348
prcp       0.001059
wspd       0.002166
pres       0.002496
tsun       1.000000
wdir       0.698620
dtype: float64


In [8]:
if RUN_PREPROCESS:
    print(aqi.isnull().mean())

time       0.0
yard_id    0.0
pm2_5      0.0
pm10       0.0
o3         0.0
co         0.0
no         0.0
no2        0.0
so2        0.0
nh3        0.0
aqhi       0.0
dtype: float64


In [9]:
if RUN_PREPROCESS:
    print(ee.isnull().mean()) # 2023 is null , need to update for simulations later...

time       0.0
yard_id    0.0
ndvi       0.0
dtype: float64


In [10]:
if RUN_PREPROCESS:
    print(ee.head())

                  time  yard_id      ndvi
0  2021-04-01 00:00:00      534 -0.192196
1  2021-05-01 00:00:00      534  0.034725
2  2021-06-01 00:00:00      534  0.256440
3  2021-07-01 00:00:00      534  0.159358
4  2021-08-01 00:00:00      534  0.230404


In [11]:
if RUN_PREPROCESS:
    data['creation_date'] = pd.to_datetime(data['creation_date'],format='mixed').dt.date
    data['death_date']= pd.to_datetime(data['death_date']).dt.date

In [12]:
if RUN_PREPROCESS:
    print(Counter(data.hid.value_counts())) # Meaning that 16 632 hives are present in two season while 38405 are only in one season 

Counter({1: 106967, 2: 16466, 3: 3127})


In [13]:
if RUN_PREPROCESS:
    ### Label creation (based on season start end)
    
    def get_state_for_season(state_log):
        
        return state_log.apply(lambda x: x['death_date'] < date(x['season']+1, START_SEASON_MONTH,START_SEASON_DAY), axis=1)
    
    def get_age_for_season(state_log):
        return state_log.apply(lambda x : abs((x['death_date'] - x['creation_date']).days), axis=1)
    
    
    data['death_next_season'] = get_state_for_season(data)
    data['hive_age_next_season'] = get_age_for_season(data)

In [14]:
data['death_next_season'].mean()

0.20989415862808145

In [15]:
data.dtypes

hid                      int64
creation_date           object
operation_id             int64
death_date              object
season                   int64
death_next_season         bool
hive_age_next_season     int64
dtype: object

In [16]:
data.operation_id.value_counts()

operation_id
193    44495
69     32987
153    18268
55     14613
167    10650
161     8070
87      5845
207     5822
210     1828
195     1591
36      1084
194      973
212      758
220      631
160      407
219      397
224      251
159      227
205      193
83       188
199        2
Name: count, dtype: int64

In [17]:
data.season.value_counts()

season
2023    107999
2022     32037
2021      9244
Name: count, dtype: int64

In [18]:
if RUN_PREPROCESS:
    # BAD OPS in 2021-2022
    data = data.loc[~((data['operation_id'] == 51) & (data['season']==2021))]
    data = data.loc[~((data['operation_id'] == 87) & (data['season']==2021))]
    data = data.loc[~((data['operation_id']==167) & (data['season']==2022))]

In [19]:
data.drop(['death_date','creation_date'],axis=1).groupby(['season','operation_id']).mean(numeric_only=True)[['death_next_season','hive_age_next_season']]

death_next_season  hive_age_next_season
season operation_id                                         
2021   36                     0.492355            411.103976
       55                     0.623086            321.406164
       69                     0.661955            350.359246
       83                     0.000000            322.000000
2022   36                     0.028646            552.833333
       55                     0.315020            559.845148
       69                     0.337974            366.455468
       83                     0.552632            399.657895
       87                     0.647992            412.726216
       153                    0.416810            389.562950
       159                    0.704348            298.669565
       160                    0.248276            325.255172
       161                    0.283731            289.159781
2023   36                     0.000000            927.525469
       55                     0.046553            724.146454
       69                     0.128808            527.610175
       83                     0.055556            835.666667
       87                     0.151279            459.447534
       153                    0.036438            588.966082
       159                    0.062500            468.455357
       160                    0.064885            498.583969
       161                    0.029163            514.239139
       167                    0.132754            350.405909
       193                    0.139117            436.991190
       194                    0.038027            314.704008
       195                    0.037712            327.542426
       199                    0.000000            437.500000
       205                    0.000000            370.248705
       207                    0.219684            280.761079
       210                    0.066193            321.088074
       212                    0.325858            234.492084
       219                    0.032746            259.450882
       220                    0.020602            247.294770
       224                    0.000000            240.972112

In [20]:
len(data)

146918

In [21]:
data.isnull().mean()

hid                     0.0
creation_date           0.0
operation_id            0.0
death_date              0.0
season                  0.0
death_next_season       0.0
hive_age_next_season    0.0
dtype: float64

### Feature eng for predictive models


### From mouvements to sensors data 

- Idea is to get the location of the hive at time X between june 1st and August 31 
- Once we have that info we simply do statistic on the sequence 


***
### One hive example

In [22]:
if RUN_PREPROCESS:
    mov['to_when'].fillna(date.today(),inplace=True) # When null still didn't moved
    mov['from_when'] = pd.to_datetime(mov['from_when'],format='mixed').dt.date
    mov['to_when'] = pd.to_datetime(mov['to_when'],format='mixed').dt.date
    
    aqi['time'] = pd.to_datetime(aqi['time'],format='mixed').dt.date
    weather['time'] = pd.to_datetime(weather['time'],format='mixed').dt.date
    ee['time'] = pd.to_datetime(ee['time'],format='mixed').dt.date

In [23]:
if RUN_PREPROCESS:
    # Only keep aqi, weather and ee for summer month 
    
    aqi = aqi.loc[pd.to_datetime(aqi['time']).dt.month.isin(FEATURES_MONTH)].reset_index(drop=True)
    weather = weather.loc[pd.to_datetime(weather['time']).dt.month.isin(FEATURES_MONTH)].reset_index(drop=True)
    ee = ee.loc[pd.to_datetime(ee['time']).dt.month.isin(FEATURES_MONTH)].reset_index(drop=True)

In [24]:
if RUN_PREPROCESS:
    one = mov.loc[mov['hive_identity_id']==21199]
    print(len(one))

14


In [25]:
if RUN_PREPROCESS:
    one = (one.assign(time = [pd.date_range(start, end) 
                       for start, end 
                       in zip(one['from_when'], one['to_when'])]
              )
       .explode('time', ignore_index = True)
    )

In [26]:
if RUN_PREPROCESS:
    one['time'] = pd.to_datetime(one['time']).dt.date

In [27]:
if RUN_PREPROCESS:
    one.head()

In [28]:
if RUN_PREPROCESS:
    one = pd.merge(one,aqi, on=["yard_id","time"])

In [29]:
if RUN_PREPROCESS:
    one['season'] = pd.to_datetime(one['time']).dt.year
    one['month'] = pd.to_datetime(one['time']).dt.month

In [30]:
if RUN_PREPROCESS:
    one[['season','month','aqhi']].groupby(["season","month"]).mean()

In [31]:
if RUN_PREPROCESS:
    print(one.head())

   hive_identity_id  report_id   from_when     to_when  user_id  yard_id  \
0             21199       5415  2021-06-23  2022-03-02      121     1034   
1             21199       5415  2021-06-23  2022-03-02      121     1034   
2             21199       5415  2021-06-23  2022-03-02      121     1034   
3             21199       5415  2021-06-23  2022-03-02      121     1034   
4             21199       5415  2021-06-23  2022-03-02      121     1034   

                submitted_at  operation_id        time      pm2_5       pm10  \
0  2021-06-23 12:37:14+00:00            55  2021-06-23   0.520500   0.916000   
1  2021-06-23 12:37:14+00:00            55  2021-06-24   3.545417   4.876667   
2  2021-06-23 12:37:14+00:00            55  2021-06-25  10.084583  10.956667   
3  2021-06-23 12:37:14+00:00            55  2021-06-26  11.185417  11.919583   
4  2021-06-23 12:37:14+00:00            55  2021-06-27   4.947083   5.618333   

          o3          co        no       no2       so2       n

### ALL hives and sensors

In [32]:
if RUN_PREPROCESS:
    mov = mov[mov['hive_identity_id'].isin(list(data.hid.unique()))].reset_index(drop=True)

In [ ]:
if RUN_PREPROCESS:
    # WARNING THIS TAKES A WHILE (5 minutes...or so)
    mov = (mov.assign(time = [pd.date_range(start,end) 
                       for start, end 
                       in zip(mov['from_when'], mov['to_when'])]
              )
       .explode('time', ignore_index = True)
    )

In [ ]:
if RUN_PREPROCESS:
    mov['time'] = pd.to_datetime(mov['time']).dt.date
    
    mov = mov.loc[pd.to_datetime(mov['time']).dt.month.isin(FEATURES_MONTH)].reset_index(drop=True)

In [ ]:
if RUN_PREPROCESS:
    merged = pd.merge(mov,aqi, on=["yard_id","time"], how="left")
    merged = pd.merge(merged,weather, on=["yard_id","time"], how="left")
    merged = pd.merge(merged,ee, on=["yard_id","time"], how="left")
    merged.sort_values("time",ascending=True,inplace=True)
    # Keep value constant if no change 
    merged.fillna(method="bfill",inplace=True)
    
    merged.head()

In [ ]:
if RUN_PREPROCESS:
    merged.isnull().mean()

### TSun and Wdir high null, just ignore them 

In [ ]:
if RUN_PREPROCESS:
    merged.drop(['tsun','wdir'],axis=1,inplace=True)

### Compute stats for each hid

In [ ]:
if RUN_PREPROCESS:
    merged['season'] = pd.to_datetime(merged['time']).dt.year

In [ ]:
if RUN_PREPROCESS:
    # THis takes 10 min or so locally
    # merged['month'] = pd.to_datetime(merged['time']).dt.month
    averages = merged.drop(['from_when','to_when','time','submitted_at'],axis=1).groupby(["hive_identity_id","season"]).mean().reset_index()
    skewed = merged.drop(['from_when','to_when','time','submitted_at'],axis=1).groupby(["hive_identity_id","season"]).agg(lambda x : skew(x)).reset_index()
    maxed = merged.drop(['from_when','to_when','time','submitted_at'],axis=1).groupby(["hive_identity_id","season"]).max().reset_index()

In [ ]:
if RUN_PREPROCESS:
    averages.rename(columns={"aqhi":"aqhi_average","prcp":"prcp_average","wspd":"wspd_average","ndvi":"ndvi_average"
                             ,"tavg":"tavg_average","o3":"o3_average","hive_identity_id":"hid"},inplace=True)
    skewed.rename(columns={"aqhi":"aqhi_skew","prcp":"prcp_skew","wspd":"wspd_skew","ndvi":"ndvi_skew"
                           ,"tavg":"tavg_skew","o3":"o3_skew","hive_identity_id":"hid"},inplace=True)
    maxed.rename(columns={"aqhi":"aqhi_max","prcp":"prcp_max","wspd":"wspd_max","ndvi":"ndvi_max"
                          ,"tavg":"tavg_max","o3":"o3_max","hive_identity_id":"hid"},inplace=True)


### Bring remote sensors with season state (merge)

In [ ]:
if RUN_PREPROCESS:
    print(len(data))
    data = pd.merge(data, averages[['season','hid','aqhi_average','prcp_average','wspd_average'
                                    ,'ndvi_average','tavg_average','o3_average']], on=["season","hid"], how="left")
    print(len(data))
    data = pd.merge(data, skewed[['season','hid','aqhi_skew','prcp_skew','wspd_skew'
                                  ,'ndvi_skew','tavg_skew','o3_skew']], on=["season","hid"], how="left")
    print(len(data))
    data = pd.merge(data, maxed[['season','hid','aqhi_max','prcp_max','wspd_max'
                                 ,'ndvi_max','tavg_max','o3_max']], on=["season","hid"], how="left")
    print(len(data))

In [ ]:
data.head()

In [ ]:
len(data)

### Add region

In [ ]:
if RUN_PREPROCESS:
    operation_meta.index = operation_meta['Operation ID']
    regions = operation_meta.to_dict(orient="dict")['Location']

In [ ]:
def try_region(x):
    try:
        return regions[str(x)].split(",")[1]
    except:
        return 'Unknown'

In [ ]:
if RUN_PREPROCESS:
    data['region'] = data['operation_id'].apply(lambda x: try_region(x))
data.groupby(['operation_id','season']).count()

### Saving preprocess so we don't need to run every time

In [ ]:
if RUN_PREPROCESS:
    data.to_csv(os.path.join(ROOT_PATH,"preprocess.csv"),index=False)
else:
    data = pd.read_csv(os.path.join(ROOT_PATH,"preprocess.csv"))

In [ ]:
len(data)

In [ ]:
ONLY_WINTER = False
ONLY_INSEASON = False 
ACTIVE_ALL_AND_WINTER = True  # Blog post hypothesis 
if ONLY_WINTER or ONLY_INSEASON:
    assert ONLY_WINTER != ONLY_INSEASON, "choose one! or both False"
if ONLY_INSEASON or ACTIVE_ALL_AND_WINTER:
    assert ACTIVE_ALL_AND_WINTER != ONLY_INSEASON, "choose one! or both False"

# Reload and apply
data = pd.read_csv(os.path.join(ROOT_PATH,"preprocess.csv"))

In [ ]:
len(data)

In [ ]:
if ONLY_WINTER:
    # Exclude in-season deadout , we are just looking at winter morta 
    data['death_date'] = pd.to_datetime(data['death_date']).dt.date
    data['winter_deadout'] = data.apply(lambda x: x['death_date'] > date(int(x['season']),END_SEASON_MONTH,END_SEASON_DAY),axis=1)
    data = data.loc[data['winter_deadout']].reset_index(drop=True)

if ONLY_INSEASON:
    data['death_date'] = pd.to_datetime(data['death_date']).dt.date
    data['winter_deadout'] = data.apply(lambda x: x['death_date'].month in [6,7,8],axis=1)
    data = data.loc[~data['winter_deadout'] | ~data['death_next_season']].reset_index(drop=True)

if ACTIVE_ALL_AND_WINTER:
    data['death_date'] = pd.to_datetime(data['death_date']).dt.date
    data['creation_date'] = pd.to_datetime(data['creation_date']).dt.date
    # This ensure the exposition rate is the full season (makes them comparable with the current approach)
    data = data.loc[(data['creation_date'] <= data['season'].apply(lambda x: date(int(x),START_SEASON_MONTH+1,START_SEASON_DAY))) \
            & (data['death_date'] > data['season'].apply(lambda x : date(int(x),END_SEASON_MONTH,END_SEASON_DAY)))]
    data = data.reset_index(drop=True)


### Keep next year aside and exclude some abnormal data

In [ ]:
next_year = data.loc[data['season']==2023].reset_index(drop=True)
if not ONLY_INSEASON:
    data = data.loc[data['season']!=2023].reset_index(drop=True)

In [ ]:
len(data)

In [ ]:
data.groupby(["death_next_season","season"]).mean(numeric_only=True)

In [ ]:
data.season.value_counts()

In [ ]:
data.isnull().mean()

*** 
### Quick viz 

Just checking a bit the data for intuition

### Check different sensors bar chart

In [ ]:
plot_survival_bar_per_state(data, "aqhi_average") # Bias by region for sure

### Check relation NDVI ~ AQHI


Idea is that increased vegetation should lead to less extreme AQHI value

In [ ]:
sns.set_palette('colorblind') # Otherwise very confusing 
sns.set_style("whitegrid")
_, ax = plt.subplots(nrows=1, ncols=1, figsize=(14,6))
for s in data['region'].unique():
    sub = data.loc[data['region']==s]
    if len(sub) > 10:
        sns.regplot(x=sub["ndvi_average"], y=sub["o3_max"], label=s, ax=ax)
plt.ylabel("AQHI Maximum value")
plt.xlabel("NDVI Average value")
plt.title("Relation between AQHI and NDVI per region")
plt.legend()
plt.show()

In [ ]:
sns.set_palette('colorblind') # Otherwise very confusing 
sns.set_style("whitegrid")
_, ax = plt.subplots(nrows=1, ncols=1, figsize=(14,6))
for s in data['region'].unique():
    sub = data.loc[data['region']==s]
    if len(sub) > 10:
        sns.regplot(x=sub["ndvi_average"], y=sub["aqhi_max"], label=s, ax=ax)
plt.ylabel("AQHI Maximum value")
plt.xlabel("NDVI Average value")
plt.title("Relation between AQHI and NDVI per region")
plt.legend()
plt.show()

### Check raw AQHI ~ mortality

Pattern for Quebec are different from the rest of the dataset

In [ ]:
sns.set_palette(NECTAR_PALETTE)
sns.set_style("whitegrid")
_, ax = plt.subplots(nrows=1, ncols=1, figsize=(14,6))
sns.regplot(x="aqhi_average",y="death_next_season",data=data,logistic=True, truncate=False, ax=ax)
plt.show()

# This is starnge and may be due to hive located norther have faced more heavy aqhi ?

In [ ]:
sns.set_palette(NECTAR_PALETTE)
sns.set_style("whitegrid")
_, ax = plt.subplots(nrows=1, ncols=1, figsize=(14,6))
sns.regplot(x="aqhi_average",y="death_next_season",data=data.loc[data['region']==" QC"],logistic=True, truncate=False, ax=ax)
plt.show()

# This is starnge and may be due to hive located norther have faced more heavy aqhi ?

In [ ]:
sns.set_palette(NECTAR_PALETTE)
sns.set_style("whitegrid")
_, ax = plt.subplots(nrows=1, ncols=1, figsize=(14,6))
sns.regplot(x="aqhi_average",y="death_next_season",data=data.loc[data['region']!=" QC"],logistic=True, truncate=False, ax=ax)
plt.show()

*** 
### Check per region

In [ ]:
data.region.value_counts()

In [ ]:
data.groupby("region").mean(numeric_only=True)

In [ ]:
# data = data.loc[data['region']!=' CA']
data.region.value_counts()

In [ ]:
sns.set_style("whitegrid")
_, axes = plt.subplots(nrows=1, ncols=1, figsize=(14,6))
ax = sns.barplot(
    y=data['aqhi_skew'],x=data["death_next_season"].apply(lambda x : "Death" if x == 1 else "Survived"), ax=axes,
    palette="blend:#D8A348,#1D1D1D"#, errcolor="darkred"
)
#  
ax.set_ylabel("Skewness AQHI")
ax.set_xlabel("Did not survived until next season")
ax.tick_params(labelrotation=90)
plt.title("Skewness AQHI \n for hives that survived or not")
plt.show()

***
### DESCRIPTIVE STATS

In [ ]:
data[['aqhi_average','ndvi_average']].isnull().mean()

In [ ]:
data.dropna(subset=['aqhi_average','ndvi_average'],axis=0,inplace=True) # Filling null value with zero

data['operation_id'] = data['operation_id'].astype(str)
data['season'] = data['season'].astype(str)

In [ ]:
len(data)

### Models 

In [ ]:
data['wspd_average_og'] = data['wspd_average']
data['tavg_average_og'] = data['tavg_average']
data['prcp_average_og'] = data['prcp_average']
data['aqhi_average_og'] = data['aqhi_average']

data['wspd_average'] = np.log(data['wspd_average']+10e-5)
data['tavg_average'] = np.log(data['tavg_average']+10e-5)
data['prcp_average'] = np.log(data['prcp_average']+10e-5)
data['aqhi_average'] = np.log(data['aqhi_average']+10e-5)

In [ ]:
sns.histplot(data['aqhi_max'])

In [ ]:
sns.histplot(data['aqhi_average'])

In [ ]:
sns.histplot(data['ndvi_average'])

In [ ]:
sns.histplot(data['wspd_average'])

In [ ]:
data.death_next_season.mean()

In [ ]:
data.isnull().mean()

In [ ]:
model0 = Lmer("death_next_season  ~ 1 + (1|region)",
             data=data, family = 'binomial')

print(model0.fit())

In [ ]:
model = Lmer("death_next_season  ~  aqhi_average*ndvi_average  + (aqhi_average|region)",
             data=data, family = 'binomial')

print(model.fit())

In [ ]:
model2 = Lmer("death_next_season  ~ aqhi_average + ndvi_average  + (aqhi_average|region)",
             data=data, family = 'binomial')

print(model2.fit())

In [ ]:
model3 = Lmer("death_next_season  ~ aqhi_average*ndvi_average  + (aqhi_average|region)",
             data=data, family = 'binomial')

print(model3.fit())

In [ ]:
model4 = Lmer("death_next_season  ~ aqhi_average + ndvi_average + tavg_average + prcp_average + wspd_average  + (aqhi_average|region)",
             data=data, family = 'binomial')

print(model4.fit())

In [ ]:
model5 = Lmer("death_next_season  ~ aqhi_average*wspd_average + ndvi_average + tavg_average + prcp_average   + (aqhi_average|region)",
             data=data, family = 'binomial')

print(model5.fit())

In [ ]:
model6 = Lmer("death_next_season  ~ aqhi_average*wspd_average*tavg_average + (aqhi_average|region)",
             data=data, family = 'binomial')

print(model6.fit())

In [ ]:
model7a = Lmer("death_next_season  ~ aqhi_average*ndvi_average*wspd_average  + (ndvi_average|region) ",
             data=data, family = 'binomial')

print(model7a.fit())

In [ ]:
model7 = Lmer("death_next_season  ~ aqhi_average*ndvi_average + wspd_average + (ndvi_average|region)",
             data=data, family = 'binomial')

print(model7.fit())

In [ ]:
model8 = Lmer("death_next_season  ~ aqhi_average*ndvi_average*wspd_average + (aqhi_average|region) ",
             data=data, family = 'binomial')

print(model8.fit())

In [ ]:
model9 = Lmer("death_next_season  ~ aqhi_skew*ndvi_average*wspd_average + (aqhi_skew|region) ",
             data=data, family = 'binomial')
print(model9.fit())

In [ ]:
model10 = Lmer("death_next_season  ~ aqhi_skew*ndvi_skew  + (aqhi_skew|region) ",
             data=data, family = 'binomial')

print(model10.fit())

In [ ]:
model11 = Lmer("death_next_season  ~ aqhi_skew*ndvi_average + wspd_average  + (aqhi_skew|region) ",
             data=data, family = 'binomial')

print(model11.fit())

In [ ]:
model12 = Lmer("death_next_season  ~ aqhi_max*ndvi_average + wspd_average  + (ndvi_average|region)",
             data=data, family = 'binomial')

print(model12.fit())

In [ ]:
model13 = Lmer("death_next_season  ~ aqhi_average*wspd_average + ndvi_average  + (ndvi_average|region)",
             data=data, family = 'binomial')

print(model13.fit())

In [ ]:
model14 = Lmer("death_next_season  ~ aqhi_max*ndvi_average*wspd_average  + (ndvi_average|region) ",
             data=data, family = 'binomial')

print(model14.fit())

In [ ]:
model15 = Lmer("death_next_season  ~ aqhi_average*ndvi_skew*wspd_average + (aqhi_average|region)",
             data=data, family = 'binomial')

print(model15.fit())

In [ ]:
model16 = Lmer("death_next_season  ~ aqhi_average*ndvi_skew + wspd_average  + (aqhi_average|region)",
             data=data, family = 'binomial')

print(model16.fit())

In [ ]:
model17 = Lmer("death_next_season  ~ aqhi_average*ndvi_skew*wspd_average  + (aqhi_average|operation_id)",
             data=data, family = 'binomial')

print(model17.fit())

In [ ]:
model18 = Lmer("death_next_season  ~ aqhi_max*ndvi_average + aqhi_average*wspd_average  + (aqhi_average|operation_id)",
             data=data, family = 'binomial')

print(model18.fit())

In [ ]:
model19 = Lmer("death_next_season  ~ ndvi_average*aqhi_average  + (aqhi_average|operation_id)",
             data=data, family = 'binomial')

print(model19.fit())

In [ ]:
model20 = Lmer("death_next_season  ~ ndvi_average*aqhi_average*wspd_average  + (aqhi_average|operation_id)",
             data=data, family = 'binomial')

print(model20.fit())

In [ ]:
model21 = Lmer("death_next_season  ~ ndvi_average*aqhi_average  + (1|operation_id)",
             data=data, family = 'binomial')

print(model21.fit())

In [ ]:
model22 = Lmer("death_next_season  ~ ndvi_average*aqhi_average*wspd_average + (1|operation_id)",
             data=data, family = 'binomial')

print(model22.fit())

In [ ]:
model23 = Lmer("death_next_season  ~ ndvi_average*aqhi_average + wspd_average + (1|operation_id)",
             data=data, family = 'binomial')

print(model23.fit())

In [ ]:
model24 = Lmer("death_next_season  ~ ndvi_average + aqhi_average + wspd_average  + (1|operation_id)",
             data=data, family = 'binomial')

print(model24.fit())

In [ ]:
model25 = Lmer("death_next_season  ~ ndvi_average + aqhi_average + wspd_average  + (aqhi_average|operation_id)",
             data=data, family = 'binomial')

print(model25.fit())

In [ ]:
model26 = Lmer("death_next_season  ~ ndvi_average + aqhi_average*wspd_average  + (aqhi_average|operation_id)",
             data=data, family = 'binomial')

print(model26.fit())

In [ ]:
model27 = Lmer("death_next_season  ~ ndvi_average + o3_average  + (o3_average|operation_id)",
             data=data, family = 'binomial')

print(model27.fit())

In [ ]:
model28 = Lmer("death_next_season  ~ ndvi_average*o3_average  + (o3_average|operation_id)",
             data=data, family = 'binomial')

print(model28.fit())

In [ ]:
model29 = Lmer("death_next_season  ~ ndvi_average*o3_average + wspd_average  + (o3_average|operation_id)",
             data=data, family = 'binomial')

print(model29.fit())

In [ ]:
model30 = Lmer("death_next_season  ~ ndvi_average*o3_average*wspd_average  + (o3_average|operation_id)",
             data=data, family = 'binomial')

print(model30.fit())

In [ ]:
model31 = Lmer("death_next_season  ~ ndvi_average + aqhi_average + wspd_average + tavg_average + prcp_average  + (aqhi_average|operation_id)",
             data=data, family = 'binomial')

print(model31.fit())

In [ ]:
model32 = Lmer("death_next_season  ~ aqhi_skew*ndvi_average*wspd_average  + (aqhi_skew|region) ",
             data=data, family = 'binomial')

print(model32.fit())

### Pick best model based on AIC

In [ ]:
i = 0
best_aic = model0.AIC
best_model = model0
for i,m in enumerate([model,model2,model3,model4,model5,model6,model7,model7a,model8,model9,model10
                      ,model11,model12,model13,model14,model15,model16,model17,model18, model19,model20, 
                      model21,model22,model23,model24,model25,model26,model27,model28,model29,model30,
                     model31,model32]):
    # Check if model converged first 
    try:
        logs = m.warnings[0]
    except:
        logs = ""
    if (len(m.warnings) == 0) | ( logs == "boundary (singular) fit: see help('isSingular')"):
        current_aic = m.AIC
        if current_aic < best_aic:
            print(f"Current best model is {i} with an AIC of {current_aic}")
            best_aic = current_aic
            best_model = m
    else:
        print(f"Model {i} did not converged")
    i+=1

In [ ]:
best_model.summary()

In [ ]:
best_model.plot_summary()

### Higher skewness , means that death hive have right skewed distribution (more positive value)

Let's check these distribution of average (which is an aggregate but should show similar pattern)

In [ ]:
sns.distplot(data.loc[data['death_next_season']]['aqhi_average_og'], label="Death", color="red")
sns.distplot(data.loc[~data['death_next_season']]['aqhi_average_og'], label="Alive",color="green")

### Shallow learning approach 

In [ ]:

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1250, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 60, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [10, 20, 40,60]
# Minimum number of samples required at each leaf node
min_samples_leaf = [10, 20, 40,60]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
features = ['aqhi_average_og','tavg_average_og','tavg_max','tavg_skew'
            ,'wspd_average_og','wspd_skew','wspd_max','ndvi_skew','ndvi_max','ndvi_average'
            ,'prcp_average_og','prcp_skew','prcp_max']
data.season.value_counts()

In [ ]:

mx=StandardScaler()

gr = pd.get_dummies(data['region'])
X = data.dropna(subset=['death_next_season','hive_age_next_season'],axis=0)
X.fillna(0,inplace=True)
X = X[features]
X = pd.concat([X,gr],axis=1)

data['death_next_season'] = data['death_next_season'].astype(bool)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    np.array(data.apply(lambda x: (x["death_next_season"], x["hive_age_next_season"]), axis=1).tolist(),dtype=[('cens', '?'), ('time', '<f8')]),
    test_size=0.15,
    random_state=8,
)

X_train = mx.fit_transform(X_train)
X_test = mx.transform(X_test)



In [ ]:

rf = RandomSurvivalForest()

rf_random = RandomizedSearchCV(
    estimator=rf,
    param_distributions=random_grid,
    n_iter=5,
    cv=3,
    verbose=1,
    random_state=8,
    n_jobs=-1,
)
# Fit the random search model
rf_random.fit(X_train, y_train)


In [ ]:
rsf = rf_random.best_estimator_

In [ ]:
print("SCORE")
print(rsf.score(X_test,y_test))

sns.displot(rsf.predict(X_test))

### Evalutate

In [ ]:
va_times = np.arange(300, 550, 7)
rsf_chf_funcs = rsf.predict_cumulative_hazard_function(
    X_test, return_array=False)
rsf_risk_scores = np.row_stack([chf(va_times) for chf in rsf_chf_funcs])

rsf_auc, rsf_mean_auc = cumulative_dynamic_auc(
    y_train, y_test, rsf_risk_scores, va_times
)


In [ ]:
plt.plot(va_times, rsf_auc, "o-", label="RSF (mean AUC = {:.3f})".format(rsf_mean_auc))
plt.xlabel("days from enrollment")
plt.ylabel("time-dependent AUC")
plt.legend(loc="lower center")
plt.savefig("ROC.png")
plt.grid(True)

### FEature importance

In [ ]:
from sklearn.inspection import permutation_importance
result = permutation_importance(
    rsf, X_test, y_test, n_repeats=15, random_state=8
)

pd.DataFrame(
    {k: result[k] for k in ("importances_mean", "importances_std",)},
    index=X.columns
).sort_values(by="importances_mean", ascending=False)

### Basic simulation

In [ ]:
def process(X, factor):
    X_copy = X.copy()
    X_copy[['aqhi_average_og']] = X[['aqhi_average_og']] * factor
    X_copy = mx.transform(X_copy)
    return X_copy
    

In [ ]:
def avg_surv_func(feat):
    surv = rsf.predict_survival_function(feat, return_array=True)
    return np.mean(surv,axis=0)

In [ ]:
X.head()

In [ ]:
preds = avg_surv_func(process(X, 1))
preds2 = avg_surv_func(process(X, 1.1))
preds3 = avg_surv_func(process(X, 1.5))
preds4 = avg_surv_func(process(X, 2))
preds8 = avg_surv_func(process(X, 4))


In [ ]:
predictions = np.stack([preds,preds2,preds3,preds4,preds8],axis=1,dtype=np.float32).flatten()
predictions.shape

In [ ]:
temps = []
for _ in [1.0,1.1,1.5,2.0,4.0]:
    temps.append([i for i in range(len(preds))])

In [ ]:
temps = np.stack(temps,axis=1).flatten()
temps.shape

In [ ]:
factor = []
for ii in [1.0,1.1,1.5,2.0,4.0]:
    f = []
    for _ in range(len(preds)):
        f.append([f"{ii}x"])
    factor.append(f)
factor = np.stack(factor,axis=1).flatten()
factor.shape

In [ ]:
results = pd.DataFrame({"Probabilité de survie":predictions,"Temps":temps, "Facteur":factor})
results.head()

In [ ]:
LANG = "EN"
title = {"FR":"Fonction de survie attendue dans des simulations \n d'intensification de mauvaise qualité de l'air",
         "EN":"Expected survival function for different simulations with varying air quality"}
xaxe = {"FR": "Temps (en jours)","EN":"Time (days)"}
yaxe = {"FR": "Probabilité de survie", "EN":"Probability of survival"}


sns.lineplot(x=results['Temps'], y=results['Probabilité de survie'], hue=results["Facteur"], linewidth=3, alpha=0.85, linestyle="solid")
plt.ylabel(yaxe[LANG])
plt.xlabel(xaxe[LANG])
plt.title(title[LANG])
plt.legend()
plt.grid(True)
plt.savefig(f"figure-2-{LANG}.png")

### Pred on 2023 true value

### Conclusion and next steps 